# Incremental margin algorithm for large margin classifiers

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.datasets import load_digits
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from scipy.stats import sem
from numpy import linalg as LA
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

## Calculating the margin

In [2]:
def compute_margin(X, y, w, b):
    margin = []
    for i in range(y.shape[0]):
        margin.append((y[i]*(np.dot(X[i,:], w)+b))/sqrt(sum(w**2))) 
    if min(margin) >= 0:
        return min(margin)
    return 0

## L0 norm

In [3]:
def L0_norm(w, threshold):
    l0_norm = 0
    for wi in w:
        if abs(wi) > threshold:
            l0_norm += 1
    return l0_norm

# ELM

In [4]:
def sigmoid(z):
    sigmoid = 1.0/(1.0 + np.exp(-z))
    return sigmoid 

## ELM with IM P inf

In [14]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin

class ELM_IMA(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000, threshold=0.1):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0
        self.threshold = threshold

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        w_norm_1 = LA.norm(w, ord=1)
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * w_norm_1 - self.lambda_param * alpha[i]):
                    if w_norm_1 != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/w_norm_1
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w - self.eta * (fixed_margin * np.sign(w) - y[i] * X[i,:])
                    w_norm_1 = LA.norm(w, ord=1)
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = np.zeros(self.H.shape[1])
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = 0#compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        l=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = LA.norm(w, ord=1)
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
            l0_norm = 0
            remove = []
            for i in range(len(self.w)):
                if abs(self.w[i]) < self.threshold * self.w.max():
                    remove.append(i)
            self.w = np.delete(self.w, remove)
            self.Z = np.delete(self.Z, remove, axis=1)
            self.H = np.delete(self.H, remove, axis=1)
            X = np.delete(X, remove, axis=1)
        return t, updates, iterations

    # Function that manage the training of IMA ELM
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-1, 1) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = sigmoid(np.dot(X, self.Z))
        #w = np.dot(np.linalg.pinv(self.H), y)  
        #self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = sigmoid(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        #else:
        #    y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## Commom ELM

In [15]:
class ELM(BaseEstimator, ClassifierMixin):
         
    def __init__(self, n_neurons):
        self.n_neurons = n_neurons
    
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-1, 1) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = sigmoid(np.dot(X, self.Z))
        self.w = np.dot(np.linalg.pinv(self.H), y)  
        return self.w, self.H, self.Z
            
    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = sigmoid(np.dot(X_new, self.Z))
        y_predicted = np.sign(np.dot(H, self.w))
        y_predicted[y_predicted==0]=1
        return y_predicted

## Function to Capture Results

In [7]:
def grid_l(X, y, p):
    # GridSearch for lambda and learning rate of IMA ELM
    parameters = {'lambda_param':np.linspace(0.01, 10, 50)}
    clf = ELM_IMA(n_neurons=p, delta_margin=10^-3, IMA_iterations=10, max_updates=10000)
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=5, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['lambda_param']

In [8]:
def results(X, y, n_splits, p, eta, IMA_iterations, lambda_param):      
    print(f'Parameters: p={p}, eta={eta}, lambda={lambda_param}')
    # Stratified k fold cross validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=72)
    i=0
    
    train_accuracy_IM_ELM_01 = np.zeros(n_splits)
    test_accuracy_IM_ELM_01 = np.zeros(n_splits)
    margin_IM_ELM_01 = np.zeros(n_splits)
    updates_01 = np.zeros(n_splits)
    iterations_FMP_01 = np.zeros(n_splits) 
    iterations_IMA_01 = np.zeros(n_splits)
    hidden_neurons_01 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_001 = np.zeros(n_splits)
    test_accuracy_IM_ELM_001 = np.zeros(n_splits)
    margin_IM_ELM_001 = np.zeros(n_splits)
    updates_001 = np.zeros(n_splits)
    iterations_FMP_001 = np.zeros(n_splits) 
    iterations_IMA_001 = np.zeros(n_splits)
    hidden_neurons_001 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_0001 = np.zeros(n_splits)
    test_accuracy_IM_ELM_0001 = np.zeros(n_splits)
    margin_IM_ELM_0001 = np.zeros(n_splits)
    updates_0001 = np.zeros(n_splits)
    iterations_FMP_0001 = np.zeros(n_splits) 
    iterations_IMA_0001 = np.zeros(n_splits)
    hidden_neurons_0001 = np.zeros(n_splits)
    
    train_accuracy_ELM = np.zeros(n_splits)
    test_accuracy_ELM = np.zeros(n_splits)
    margin_ELM = np.zeros(n_splits)
        
    for train_index, test_index in kf.split(X, y):
        X_train = X[train_index,:]
        X_test = X[test_index,:]
        y_train = y[train_index]
        y_test = y[test_index]
               
        # ELM
        clf = ELM(n_neurons=p)
        clf.fit(X_train, y_train)
        y_hat=clf.predict(X_test)
        y_hat_train=clf.predict(X_train)
        margin_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w, 0)
        train_accuracy_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_ELM[i] = accuracy_score(y_test, y_hat)
        Z = clf.Z
        
        # ELM-IMA 0.1
        clf_01 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.1)
        iterations_IMA_01[i], updates_01[i], iterations_FMP_01[i] = clf_01.fit(X_train, y_train, Z)
        y_hat=clf_01.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_01.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_01[i] = compute_margin(clf_01.H[:,:], y_train, clf_01.w, clf_01.b)
        train_accuracy_IM_ELM_01[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_01[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_01[i] = len(clf_01.w)
        
                
        # ELM-IMA 0.01
        clf_001 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.01)
        iterations_IMA_001[i], updates_001[i], iterations_FMP_001[i] = clf_001.fit(X_train, y_train, Z)
        y_hat=clf_001.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_001.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_001[i] = compute_margin(clf_001.H[:,:], y_train, clf_001.w, clf_001.b)
        train_accuracy_IM_ELM_001[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_001[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_001[i] = len(clf_001.w)

        # ELM-IMA 0.001
        clf_0001 = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.001)
        iterations_IMA_0001[i], updates_0001[i], iterations_FMP_0001[i] = clf_0001.fit(X_train, y_train, Z)
        y_hat=clf_0001.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_0001.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_0001[i] = compute_margin(clf_0001.H[:,:], y_train, clf_0001.w, clf_0001.b)
        train_accuracy_IM_ELM_0001[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_0001[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_0001[i] = len(clf_0001.w)       
        i+=1
        
    print("********* Results ELM-IMA 0.1**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_01.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_01.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_01.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_01.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_01.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_01.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_01.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_01.std()))
    print("Updates: " + '{:.4f}'.format(updates_01.mean())+ "+/-" + '{:.4f}'.format(updates_01.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_01.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_01.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_01.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_01.std()))
    
    print("********* Results ELM-IMA 0.01**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_001.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_001.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_001.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_001.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_001.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_001.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_001.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_001.std()))
    print("Updates: " + '{:.4f}'.format(updates_001.mean())+ "+/-" + '{:.4f}'.format(updates_001.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_001.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_001.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_001.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_001.std()))
    
    print("********* Results ELM-IMA 0.001**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_0001.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_0001.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_0001.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_0001.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_0001.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_0001.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_0001.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_0001.std()))
    print("Updates: " + '{:.4f}'.format(updates_0001.mean())+ "+/-" + '{:.4f}'.format(updates_0001.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_0001.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_0001.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_0001.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_0001.std()))
            
    print("********* Results ELM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_ELM.std()))
    print("Margin: " + '{:.9f}'.format(margin_ELM.mean())+  "+/-" + '{:.9f}'.format(margin_ELM.std()))  

In [9]:
def run(X,y):
    # Normalizing data:
    normalizer = preprocessing.Normalizer()
    X = normalizer.fit_transform(X)
    n = len(X)
    if n>1000:
        n=1000
    for p in [n]:
        l = grid_l(X, y, p=p)
        print(f"Experimento com {p} neurônios:" )
        results(X, y, n_splits=10, p=p, eta=0.1, IMA_iterations=20, lambda_param=l)
        print("\n\n")

## Application on Iris Dataset


In [10]:
iris = datasets.load_iris()
X = iris.data
# setosa - 0, versicolor - 1, virginica - 2  
y = iris.target 
# O problema agora possui apenas as classes y=-1 e y=1
y[y>0] = 1
y[y==0] = -1
run(X, y)

Experimento com 150 neurônios:
Parameters: p=150, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 1232.9000+/-879.9007
Iterations IMA: 20.0000+/-0.0000
Updates: 3609.7000+/-1683.3828
Margin: 0.116311370+/-0.017844497
Hidden Neurons: 20.60+/-5.66
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 1234.8000+/-539.9957
Iterations IMA: 20.0000+/-0.0000
Updates: 4547.4000+/-1149.3676
Margin: 0.128129018+/-0.008188443
Hidden Neurons: 27.40+/-9.24
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 1341.9000+/-1018.4696
Iterations IMA: 20.0000+/-0.0000
Updates: 5492.4000+/-3222.2697
Margin: 0.133863605+/-0.012691819
Hidden Neurons: 79.40+/-21.48
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Margin: 0.000000000+/-0.000000000





## Application on Synthetic Dataset

In [11]:
synthetic_dataset = pd.read_csv('data/synthetic_dataset/synthetic_control.data', sep="\s+",  header=None, engine='python')
X = synthetic_dataset.to_numpy()
y = np.concatenate((np.ones(100), np.ones(200)*-1, np.ones(100), np.ones(100)*-1,np.ones(100)))
run(X, y)

Experimento com 600 neurônios:
Parameters: p=600, eta=0.1, lambda=2.2526530612244895
********* Results ELM-IMA 0.1**************
Acc train: 0.9961+/-0.0063
Acc test: 0.9867+/-0.0233
Iterations: 201.8000+/-112.9733
Iterations IMA: 20.0000+/-0.0000
Updates: 3730.5000+/-2037.2679
Margin: 0.001476860+/-0.002655616
Hidden Neurons: 345.40+/-54.62
********* Results ELM-IMA 0.01**************
Acc train: 0.9952+/-0.0044
Acc test: 0.9750+/-0.0403
Iterations: 176.7000+/-65.9273
Iterations IMA: 20.0000+/-0.0000
Updates: 3425.0000+/-1651.8901
Margin: 0.002834857+/-0.004935276
Hidden Neurons: 498.80+/-22.63
********* Results ELM-IMA 0.001**************
Acc train: 0.9972+/-0.0030
Acc test: 0.9850+/-0.0345
Iterations: 216.8000+/-111.5480
Iterations IMA: 20.0000+/-0.0000
Updates: 5062.5000+/-3864.4110
Margin: 0.000829275+/-0.001304060
Hidden Neurons: 549.50+/-56.61
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7067+/-0.0638
Margin: 0.000131400+/-0.000012691





## Application on Robot  Dataset



In [12]:
robot_dataset = pd.read_csv('data/robot/lp4_data.csv', delimiter =',')
X = robot_dataset.to_numpy().reshape([117,90])
y = np.concatenate((np.ones(24), np.ones(117-24)*-1))
run(X, y)

Experimento com 117 neurônios:
Parameters: p=117, eta=0.1, lambda=8.572857142857142
********* Results ELM-IMA 0.1**************
Acc train: 0.9915+/-0.0067
Acc test: 0.9833+/-0.0500
Iterations: 79.1000+/-17.7620
Iterations IMA: 20.0000+/-0.0000
Updates: 237.9000+/-73.8816
Margin: 0.012204205+/-0.023850406
Hidden Neurons: 44.10+/-13.88
********* Results ELM-IMA 0.01**************
Acc train: 0.9962+/-0.0047
Acc test: 0.9750+/-0.0534
Iterations: 76.4000+/-16.7404
Iterations IMA: 20.0000+/-0.0000
Updates: 246.3000+/-78.5825
Margin: 0.049671701+/-0.058164843
Hidden Neurons: 71.90+/-13.34
********* Results ELM-IMA 0.001**************
Acc train: 0.9924+/-0.0093
Acc test: 0.9917+/-0.0250
Iterations: 86.4000+/-30.8843
Iterations IMA: 20.0000+/-0.0000
Updates: 327.1000+/-162.8695
Margin: 0.026076210+/-0.031512028
Hidden Neurons: 100.20+/-8.49
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.8432+/-0.1589
Margin: 0.016406460+/-0.003271136





## Application on Mushroom Dataset

In [28]:
df = pd.read_csv('data/Mushroom/agaricus-lepiota.data', delimiter =',', header=None)
df = df.replace("?", np.nan) 
df = df.dropna() 
y = df[0].to_numpy()
X = df.drop([0], axis='columns')
X = pd.get_dummies(X).to_numpy()
y[np.where(y=='e')] = -1
y[np.where(y=='p')] = 1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=8.572857142857142
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9816+/-0.0553
Iterations: 83.4000+/-33.1488
Iterations IMA: 20.0000+/-0.0000
Updates: 1550.9000+/-453.4778
Margin: 0.161095918+/-0.046215993
Hidden Neurons: 512.90+/-103.67
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9816+/-0.0553
Iterations: 87.5000+/-39.3961
Iterations IMA: 20.0000+/-0.0000
Updates: 1712.0000+/-859.5726
Margin: 0.142506384+/-0.059292175
Hidden Neurons: 781.70+/-119.52
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9816+/-0.0553
Iterations: 84.3000+/-40.3387
Iterations IMA: 20.0000+/-0.0000
Updates: 1769.1000+/-1181.3681
Margin: 0.130323204+/-0.057381014
Hidden Neurons: 894.90+/-122.84
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9821+/-0.0537
Margin: 0.124361135+/-0.011935599





## Application on Ionosphere Dataset

In [16]:
ionosphere_dataset = pd.read_csv('data/Ionosphere/ionosphere.data', names=list(range(0,35)), sep=',')
y = ionosphere_dataset[34].to_numpy()
X = ionosphere_dataset.drop([34], axis='columns').to_numpy()
y[np.where(y=='g')] = 1
y[np.where(y=='b')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 351 neurônios:
Parameters: p=351, eta=0.1, lambda=7.349591836734694
********* Results ELM-IMA 0.1**************
Acc train: 0.9452+/-0.0100
Acc test: 0.9117+/-0.0449
Iterations: 212.9000+/-51.8969
Iterations IMA: 20.0000+/-0.0000
Updates: 2576.8000+/-254.5835
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 135.90+/-28.44
********* Results ELM-IMA 0.01**************
Acc train: 0.9490+/-0.0139
Acc test: 0.9145+/-0.0495
Iterations: 223.8000+/-87.8679
Iterations IMA: 20.0000+/-0.0000
Updates: 3011.5000+/-775.5004
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 238.80+/-35.85
********* Results ELM-IMA 0.001**************
Acc train: 0.9405+/-0.0230
Acc test: 0.8918+/-0.0521
Iterations: 247.3000+/-109.1055
Iterations IMA: 20.0000+/-0.0000
Updates: 3626.0000+/-1071.1322
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 315.40+/-21.50
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7467+/-0.0828
Margin: 0.000801092+/-0.000294789





## Application on Banknote Dataset

In [17]:
# read in banknote authentication set
banknotes = pd.read_csv('data/banknote/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])
# convert to array
X = banknotes[['variance', 'skewness', 'curtosis', 'entropy']].to_numpy()
y = banknotes[['class']].to_numpy()
y[np.where(y==0)] = -1
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.1**************
Acc train: 0.9550+/-0.1332
Acc test: 0.9533+/-0.1329
Iterations: 2244.6000+/-951.7887
Iterations IMA: 16.0000+/-7.3075
Updates: 20498.3000+/-7920.7612
Margin: 0.028867533+/-0.018952747
Hidden Neurons: 477.10+/-229.11
********* Results ELM-IMA 0.01**************
Acc train: 0.9555+/-0.1334
Acc test: 0.9540+/-0.1331
Iterations: 1955.9000+/-1202.2978
Iterations IMA: 16.9000+/-6.3945
Updates: 18799.7000+/-7779.4331
Margin: 0.044073148+/-0.016603311
Hidden Neurons: 747.50+/-129.17
********* Results ELM-IMA 0.001**************
Acc train: 0.9552+/-0.1333
Acc test: 0.9533+/-0.1329
Iterations: 1861.8000+/-866.9453
Iterations IMA: 14.7000+/-7.4169
Updates: 19662.8000+/-6632.1591
Margin: 0.037132139+/-0.020741476
Hidden Neurons: 880.80+/-85.84
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.9322+/-0.0176
Margin: 0.000000000+/-0.000000000





## Application on Wine Dataset

In [18]:
wine_dataset = pd.read_csv('data/wine/wine.data', names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline'])
# convert to array
y = wine_dataset[['Class']].to_numpy()
X = wine_dataset.drop("Class",axis='columns').to_numpy()
y[np.where(y==3)] = 1
y[np.where(y==2)] = -1
run(X, y)

Experimento com 178 neurônios:
Parameters: p=178, eta=0.1, lambda=0.8255102040816327
********* Results ELM-IMA 0.1**************
Acc train: 0.6916+/-0.0427
Acc test: 0.7137+/-0.1474
Iterations: 568.5000+/-131.5236
Iterations IMA: 18.4000+/-4.8000
Updates: 13164.0000+/-2863.1405
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 53.90+/-25.64
********* Results ELM-IMA 0.01**************
Acc train: 0.7029+/-0.0568
Acc test: 0.7095+/-0.1368
Iterations: 619.0000+/-222.9910
Iterations IMA: 18.4000+/-4.8000
Updates: 15635.3000+/-5066.6394
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 89.30+/-34.58
********* Results ELM-IMA 0.001**************
Acc train: 0.6928+/-0.1073
Acc test: 0.6987+/-0.1917
Iterations: 688.1000+/-238.9081
Iterations IMA: 18.4000+/-4.8000
Updates: 20009.7000+/-6253.8259
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 119.90+/-33.16
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6748+/-0.1177
Margin: 0.000000001+/-0.000000000





## Application on WDBC Dataset

In [19]:
wdbc_dataset = pd.read_csv('data/WDBC/wdbc.data', names=list(range(0,32)))
# convert to array
y = wdbc_dataset[1].to_numpy()
X = wdbc_dataset.drop([0, 1],axis='columns').to_numpy()
y[np.where(y=='B')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 569 neurônios:
Parameters: p=569, eta=0.1, lambda=7.75734693877551
********* Results ELM-IMA 0.1**************
Acc train: 0.9012+/-0.0192
Acc test: 0.8981+/-0.0449
Iterations: 319.3000+/-250.7720
Iterations IMA: 20.0000+/-0.0000
Updates: 7171.4000+/-3627.0806
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 312.90+/-56.40
********* Results ELM-IMA 0.01**************
Acc train: 0.8834+/-0.0380
Acc test: 0.8788+/-0.0539
Iterations: 403.4000+/-395.2175
Iterations IMA: 20.0000+/-0.0000
Updates: 9620.4000+/-4753.6439
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 424.80+/-62.61
********* Results ELM-IMA 0.001**************
Acc train: 0.8819+/-0.0435
Acc test: 0.8717+/-0.0509
Iterations: 600.1000+/-472.9377
Iterations IMA: 20.0000+/-0.0000
Updates: 12070.6000+/-6093.5300
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 494.20+/-56.73
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7083+/-0.0836
Margin: 0.000000001+/-0.000000000





## Application on Sonar Dataset

In [20]:
sonar_dataset = pd.read_csv('data/sonar/sonar.all-data', names=list(range(0,61)), sep=',')
y = sonar_dataset[60].to_numpy()
X = sonar_dataset.drop([60], axis='columns').to_numpy()
y[np.where(y=='R')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 208 neurônios:
Parameters: p=208, eta=0.1, lambda=5.514693877551021
********* Results ELM-IMA 0.1**************
Acc train: 0.7606+/-0.0735
Acc test: 0.6207+/-0.1455
Iterations: 237.5000+/-74.4302
Iterations IMA: 20.0000+/-0.0000
Updates: 4192.3000+/-811.7347
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 73.10+/-22.96
********* Results ELM-IMA 0.01**************
Acc train: 0.7821+/-0.0728
Acc test: 0.6588+/-0.1154
Iterations: 281.0000+/-110.6689
Iterations IMA: 20.0000+/-0.0000
Updates: 5474.1000+/-1427.1387
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 116.20+/-35.13
********* Results ELM-IMA 0.001**************
Acc train: 0.7816+/-0.0712
Acc test: 0.6205+/-0.1584
Iterations: 283.2000+/-73.2350
Iterations IMA: 20.0000+/-0.0000
Updates: 5933.7000+/-1677.3815
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 175.70+/-17.10
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5874+/-0.1322
Margin: 0.000522685+/-0.000110384





## Application on Pima Dataset

In [21]:
pima_dataset = pd.read_csv('data/diabetes/diabetes.csv', sep=",", engine='python')
y = pima_dataset['Outcome'].to_numpy()
X = pima_dataset.drop(['Outcome'], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 768 neurônios:
Parameters: p=768, eta=0.1, lambda=2.0487755102040817
********* Results ELM-IMA 0.1**************
Acc train: 0.6549+/-0.0093
Acc test: 0.6510+/-0.0193
Iterations: 107.8000+/-148.3265
Iterations IMA: 1.5000+/-0.6708
Updates: 13332.6000+/-5485.2641
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 680.10+/-115.61
********* Results ELM-IMA 0.01**************
Acc train: 0.6515+/-0.0032
Acc test: 0.6523+/-0.0196
Iterations: 43.4000+/-18.8478
Iterations IMA: 1.4000+/-0.4899
Updates: 12402.0000+/-3023.3864
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 762.40+/-7.13
********* Results ELM-IMA 0.001**************
Acc train: 0.6513+/-0.0030
Acc test: 0.6523+/-0.0196
Iterations: 43.3000+/-18.7406
Iterations IMA: 1.4000+/-0.4899
Updates: 12379.8000+/-2999.4686
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 767.30+/-1.00
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.4857+/-0.0491
Margin: 0.000000000+/-0.000000000





## Application on Statlog Dataset

In [22]:
statlog_dataset = pd.read_csv('data/statlog/heart.dat', sep=" ", header=None, engine='python')
y = statlog_dataset[13].to_numpy()
X = statlog_dataset.drop([13], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 270 neurônios:
Parameters: p=270, eta=0.1, lambda=0.6216326530612245
********* Results ELM-IMA 0.1**************
Acc train: 0.7000+/-0.0392
Acc test: 0.6963+/-0.1018
Iterations: 606.3000+/-184.9768
Iterations IMA: 20.0000+/-0.0000
Updates: 31485.4000+/-11615.5041
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 57.30+/-21.54
********* Results ELM-IMA 0.01**************
Acc train: 0.6827+/-0.0894
Acc test: 0.6778+/-0.1273
Iterations: 683.9000+/-430.4780
Iterations IMA: 18.8000+/-3.6000
Updates: 42281.8000+/-28400.8517
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 156.60+/-44.01
********* Results ELM-IMA 0.001**************
Acc train: 0.6329+/-0.0915
Acc test: 0.6519+/-0.1138
Iterations: 582.6000+/-406.9688
Iterations IMA: 14.4000+/-7.0456
Updates: 36303.4000+/-24429.7366
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 233.70+/-33.64
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5889+/-0.1400
Margin: 0.000000016+/-0.000000003





## Application on Mammographic Dataset

In [23]:
mammo = pd.read_csv('data/mammographic/mammographic_masses.data', sep=",", header=None, engine='python')
mammo = mammo.replace("?", np.nan)
mammo = mammo.dropna()
y = mammo[5].to_numpy()
X = mammo.drop([5], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 830 neurônios:
Parameters: p=830, eta=0.1, lambda=7.75734693877551
********* Results ELM-IMA 0.1**************
Acc train: 0.6075+/-0.0814
Acc test: 0.6000+/-0.0890
Iterations: 614.3000+/-425.3975
Iterations IMA: 18.1000+/-5.7000
Updates: 38327.4000+/-15368.3756
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 343.70+/-189.68
********* Results ELM-IMA 0.01**************
Acc train: 0.6050+/-0.0901
Acc test: 0.6133+/-0.1170
Iterations: 588.5000+/-438.0580
Iterations IMA: 15.7000+/-7.0434
Updates: 52686.8000+/-26593.0050
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 483.40+/-199.58
********* Results ELM-IMA 0.001**************
Acc train: 0.5973+/-0.0908
Acc test: 0.5904+/-0.1002
Iterations: 394.5000+/-273.4086
Iterations IMA: 15.0000+/-7.7201
Updates: 44940.0000+/-27767.1665
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 699.40+/-69.60
********* Results ELM **************
Acc train: 0.8869+/-0.0051
Acc test: 0.7614+/-0.0420
Margin: 0.000000000+/-0.000000000





## Application on Haberman Dataset

In [24]:
haberman = pd.read_csv('data/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 306 neurônios:
Parameters: p=306, eta=0.1, lambda=0.8255102040816327
********* Results ELM-IMA 0.1**************
Acc train: 0.7051+/-0.1045
Acc test: 0.7290+/-0.0488
Iterations: 310.0000+/-124.9328
Iterations IMA: 4.8000+/-3.4583
Updates: 20784.5000+/-2495.6583
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 46.70+/-28.17
********* Results ELM-IMA 0.01**************
Acc train: 0.7357+/-0.0021
Acc test: 0.7353+/-0.0094
Iterations: 363.3000+/-330.9514
Iterations IMA: 5.2000+/-4.3543
Updates: 24751.2000+/-9786.5257
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 182.40+/-59.83
********* Results ELM-IMA 0.001**************
Acc train: 0.7357+/-0.0016
Acc test: 0.7353+/-0.0094
Iterations: 261.4000+/-81.7829
Iterations IMA: 3.8000+/-1.2490
Updates: 21789.1000+/-4561.9412
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 270.10+/-30.22
********* Results ELM **************
Acc train: 0.8315+/-0.0069
Acc test: 0.7286+/-0.0373
Margin: 0.000000000+/-0.000000000





## Application on Transfusion Dataset

In [25]:
transfusion = pd.read_csv('data/transfusion/transfusion.data', sep=",", engine='python')
y = transfusion["whether he/she donated blood in March 2007"].to_numpy()
X = transfusion.drop(["whether he/she donated blood in March 2007"], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 748 neurônios:
Parameters: p=748, eta=0.1, lambda=0.41775510204081634
********* Results ELM-IMA 0.1**************
Acc train: 0.7044+/-0.0917
Acc test: 0.7021+/-0.1594
Iterations: 129.2000+/-116.3450
Iterations IMA: 1.3000+/-0.4583
Updates: 10451.9000+/-484.4080
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 708.90+/-59.73
********* Results ELM-IMA 0.01**************
Acc train: 0.7430+/-0.0359
Acc test: 0.7034+/-0.1589
Iterations: 127.9000+/-116.0771
Iterations IMA: 1.3000+/-0.4583
Updates: 10463.1000+/-484.1238
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 744.40+/-6.17
********* Results ELM-IMA 0.001**************
Acc train: 0.7436+/-0.0339
Acc test: 0.7034+/-0.1589
Iterations: 127.9000+/-116.0771
Iterations IMA: 1.3000+/-0.4583
Updates: 10464.9000+/-487.3183
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 747.70+/-0.46
********* Results ELM **************
Acc train: 0.7947+/-0.0112
Acc test: 0.7327+/-0.1153
Margin: 0.000000000+/-0.000000000





## Application on Australian Credit 

In [26]:
australian = pd.read_csv('data/australian_credit/australian.dat', header=None, sep=" ", engine='python')
australian = australian.replace("?", np.nan)
australian = australian.dropna()
y = australian[14].to_numpy()
X = australian.drop([14], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 690 neurônios:
Parameters: p=690, eta=0.1, lambda=7.961224489795918
********* Results ELM-IMA 0.1**************
Acc train: 0.7097+/-0.0483
Acc test: 0.7261+/-0.0603
Iterations: 523.2000+/-146.0327
Iterations IMA: 20.0000+/-0.0000
Updates: 30883.5000+/-6761.8284
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 252.40+/-38.50
********* Results ELM-IMA 0.01**************
Acc train: 0.7403+/-0.0348
Acc test: 0.7536+/-0.0506
Iterations: 492.9000+/-162.4078
Iterations IMA: 20.0000+/-0.0000
Updates: 33415.4000+/-6299.7893
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 460.20+/-74.60
********* Results ELM-IMA 0.001**************
Acc train: 0.7501+/-0.0162
Acc test: 0.7580+/-0.0523
Iterations: 508.7000+/-148.7293
Iterations IMA: 20.0000+/-0.0000
Updates: 38597.2000+/-10497.8891
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 647.90+/-11.34
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.5623+/-0.0691
Margin: 0.000000002+/-0.000000000





## Application on Breast Cancer Dataset

In [27]:
breast = pd.read_csv('data/breast/breast.data', header=None, sep=",", engine='python')
breast = breast.replace("?", np.nan)
breast = breast.dropna()
y = breast[10].to_numpy()
X = breast.drop([0, 10], axis='columns').to_numpy()
y[np.where(y==4)] = 1
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 683 neurônios:
Parameters: p=683, eta=0.1, lambda=2.660408163265306
********* Results ELM-IMA 0.1**************
Acc train: 0.8640+/-0.0208
Acc test: 0.8332+/-0.0651
Iterations: 377.4000+/-70.8099
Iterations IMA: 20.0000+/-0.0000
Updates: 22727.3000+/-2813.0048
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 48.10+/-28.88
********* Results ELM-IMA 0.01**************
Acc train: 0.8388+/-0.0343
Acc test: 0.8289+/-0.0632
Iterations: 649.3000+/-202.3384
Iterations IMA: 18.2000+/-5.4000
Updates: 42812.5000+/-12703.4053
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 151.10+/-178.74
********* Results ELM-IMA 0.001**************
Acc train: 0.8206+/-0.0331
Acc test: 0.8142+/-0.0593
Iterations: 1306.9000+/-179.2738
Iterations IMA: 20.0000+/-0.0000
Updates: 93392.1000+/-15024.8907
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 332.60+/-74.11
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.7851+/-0.0602
Margin: 0.000010104+/-0.000000816





## Application on Spam Dataset

In [30]:
spam = pd.read_csv('data/spam/spambase.data', header=None, sep=",", engine='python')
y = spam[57].to_numpy()
X = spam.drop([57], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=8.572857142857142
********* Results ELM-IMA 0.1**************
Acc train: 0.6060+/-0.0001
Acc test: 0.6060+/-0.0009
Iterations: 161.7000+/-129.8908
Iterations IMA: 1.0000+/-0.0000
Updates: 10240.9000+/-222.9293
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 1000.00+/-0.00
********* Results ELM-IMA 0.01**************
Acc train: 0.6060+/-0.0001
Acc test: 0.6060+/-0.0009
Iterations: 161.7000+/-129.8908
Iterations IMA: 1.0000+/-0.0000
Updates: 10240.9000+/-222.9293
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 1000.00+/-0.00
********* Results ELM-IMA 0.001**************
Acc train: 0.6060+/-0.0001
Acc test: 0.6060+/-0.0009
Iterations: 161.7000+/-129.8908
Iterations IMA: 1.0000+/-0.0000
Updates: 10240.9000+/-222.9293
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 1000.00+/-0.00
********* Results ELM **************
Acc train: 0.9283+/-0.0042
Acc test: 0.8229+/-0.0493
Margin: 0.000000000+/-0.000000000





## Application on Glass Dataset

In [29]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1
run(X, y)

Experimento com 214 neurônios:
Parameters: p=214, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.1**************
Acc train: 0.6630+/-0.0300
Acc test: 0.6364+/-0.1063
Iterations: 118.8000+/-34.8362
Iterations IMA: 1.2000+/-0.6000
Updates: 10561.4000+/-1462.7818
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 207.30+/-20.10
********* Results ELM-IMA 0.01**************
Acc train: 0.6719+/-0.0035
Acc test: 0.6773+/-0.0180
Iterations: 114.8000+/-23.2370
Iterations IMA: 1.1000+/-0.3000
Updates: 10326.4000+/-758.0130
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 213.60+/-1.20
********* Results ELM-IMA 0.001**************
Acc train: 0.6719+/-0.0035
Acc test: 0.6773+/-0.0180
Iterations: 114.8000+/-23.2370
Iterations IMA: 1.1000+/-0.3000
Updates: 10320.2000+/-739.4250
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 214.00+/-0.00
********* Results ELM **************
Acc train: 1.0000+/-0.0000
Acc test: 0.6165+/-0.1249
Margin: 0.000000000+/-0.000000000





In [39]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=72)
for train_index, test_index in kf.split(X, y):
    print(test_index)

[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29   30   31   32   33   34   35   36   37   38   39   40   41
   42   43   44   45   46   47   48   49   50   51   52   53   54   55
   56   57   58   59   60   61   62   63   64   65   66   67   68   69
   70   71   72   73   74   75   76   77   78   79   80   81   82   83
   84   85   86   87   88   89   90   91   92   93   94   95   96   97
   98   99  100  101  102  103  104  105  106  107  108  109  110  111
  112  113  114  115  116  117  118  119  120  121  122  123  124  125
  126  127  128  129  130  131  132  133  134  135  136  137  138  139
  140  141  142  143  144  145  146  147  148  149  150  151  152  153
  154  155  156  157  158  159  160  161  162  163  164  165  166  167
  168  169  170  171  172  173  174  175  176  177  178  179  180  181
 1813 1814 1815 1816 1817 1818 1819 1820 1821 1822 1823 1824 1825 1826
 1827 